# Este código tem como objetivo a tentativa da criação de campeonatos no fantasy game Cartola FC

In [2]:
#Importando as bibliotecas necessárias
import json
import requests
import re
import numpy as np
import pandas as pd
import openpyxl

# Extração e Limpeza dos dados da api do Cartola FC

In [3]:
#Selecionar a rodada que será analisada, qual a forma de busca e quantos times
rodada = input("Qual rodada você deseja buscar? ")
id_nome = input("Deseja colocar diretamente os ids dos times? Sim ou Não? ")
n_times = input("Quantos times? ")

Qual rodada você deseja buscar? 30
Deseja colocar diretamente os ids dos times? Sim ou Não? Sim
Quantos times? 32


In [5]:
#5082775;3253808;18726872;20594620;5737183;7880178;25293386;560070;7946153;18286660;25270804;26311943;24920869;44570930;26118034;2079745;1322672;386656;308589;993764;19560070;27836036;25488938;14111267;44784735;580912;1545128;1287359;15637218;44616210;44611579;8772457

In [12]:
#Criando a lista de times participando do campeonato
contagem = 0
ids = []
lista_times = []
lista = input("Tem a lista de ids de algum app? Sim ou Não? ")
if lista == "Sim":
    lista_id = input("Cole a lista aqui (apenas os números): ")
    lista_id = lista_id.split(";")
    ids = lista_id
else:
    
    if id_nome == "Sim":
        while contagem < int(n_times):
            contagem = contagem + 1
            novo_id = input("Coloque o id: ")
            ids.append(novo_id)
    else:
        while contagem < int(n_times):
            contagem = contagem + 1
            novo_time = input("Coloque o nome do time: ")
            lista_times.append(novo_time)
#Obs.: É necessário verificar se o nome leva diretamente ai time procurado, 
#caso tenha dificuldades é interessante colocar os ids diretamente

Tem a lista de ids de algum app? Sim ou Não? Sim
Cole a lista aqui (apenas os números): 5082775;3253808;18726872;20594620;5737183;7880178;25293386;560070;7946153;18286660;25270804;26311943;24920869;44570930;26118034;2079745;1322672;386656;308589;993764;19560070;27836036;25488938;14111267;44784735;580912;1545128;1287359;15637218;44616210;44611579;8772457


In [13]:
#Caso a lista inserida seja dos nomes dos times é necessário extrair os ids da api
if id_nome == "Não":
    id_times = lista_times
    a = 0
#Importando e limpando os ids dos times listados acima
    for x in lista_times:
        id_times = requests.get("https://api.cartola.globo.com/times?q=" + x)
        id_times = id_times.content
        id_times = str(id_times)
        id_times = id_times.split(",")
        id_times = id_times[7]
        id_times = id_times.split(":")
        id_times = id_times[1]
        ids.append(id_times)
        a = a + 1
else:
    print("Prosseguir com extração das pontuações :)")

Prosseguir com extração das pontuações :)


In [14]:
#Caso o time não tenha sido escalado no campeonato, o id será um valor aleatório
print(ids)

['5082775', '3253808', '18726872', '20594620', '5737183', '7880178', '25293386', '560070', '7946153', '18286660', '25270804', '26311943', '24920869', '44570930', '26118034', '2079745', '1322672', '386656', '308589', '993764', '19560070', '27836036', '25488938', '14111267', '44784735', '580912', '1545128', '1287359', '15637218', '44616210', '44611579', '8772457']


In [15]:
#Caso haja algum problema de não retornar a pontuação do time é possível verificar os ids diretamente na api
verificar = input("Deseja verificar erro de algum time? Sim ou Não?\n")
if verificar == "Sim":
    id_verificar = input("Qual o nome do time?\n")
    print("\n \nDados do time importado:\n")
    info_time = requests.get("https://api.cartola.globo.com/times?q=" + id_verificar)
    print(info_time.content)
elif verificar == "Não":
    print("Ok, seguir com a limpeza dos dados")
else:
    print("Esta não é uma resposta válida")

Deseja verificar erro de algum time? Sim ou Não?
Não
Ok, seguir com a limpeza dos dados


### Caso você deseja trocar apenas a rodada, você pode pular direto pra cá alterando apenas a rodada e mantendo os valores dos times

In [1]:
pontos_rodada = []
times = ""
#Importando os dados do time na rodada selecionada e buscando a pontuação dele na rodada
for i in ids:
    times = requests.get("https://api.cartola.globo.com/time/id/" + i + "/" + rodada)
    times = times.content
    times = str(times)
    times = times.split(",")
    times = times[-12]
    times = times.split(":")
    times = times[1]
    pontos_rodada.append(times)
    

NameError: name 'ids' is not defined

In [17]:
#Transformando em DataFrame
pd_pontos_rodada = pd.DataFrame(pontos_rodada)
pd_pontos_rodada = pd_pontos_rodada.T
pd_pontos_rodada.columns = [ids]
pd_pontos_rodada.index = ["Rodada " + rodada]
pd_pontos_rodada = pd_pontos_rodada.replace(to_replace = 'null', value = '0')
pd_pontos_rodada = pd_pontos_rodada.astype(float)

display(pd_pontos_rodada)

,5082775,3253808,18726872,20594620,5737183,7880178,25293386,560070,7946153,18286660,...,25488938,14111267,44784735,580912,1545128,1287359,15637218,44616210,44611579,8772457
Rodada 30,68.819824,101.819824,81.02002,51.169922,27.150024,107.02002,0.0,47.149902,53.52002,88.319824,...,65.350098,70.47998,89.720215,93.620117,44.649902,94.319824,74.620117,53.850098,100.930176,35.22998


In [10]:
#Caso haja algum erro em relação às pontuações, verificar se o nome do time retorna corretamente os ids na linha anterior.
#Se as informações não retornarem corretamente eu sugiro verificar se nome do time no próprio cartola não retorna
#algum outro time. É possível tentar encontrar o time pelo nome do dono também

# Integração com o Excel

In [11]:
#Esse código foi criado com auxílio do vídeo "https://www.youtube.com/watch?v=IT7zPluDADk&t=625s"

In [19]:
#Importando a tabela que será utilizada
#Recomendo o uso do openpyxl caso aconteça algum erro na importação
path = "C://Users//Família//Desktop//Ian//cartoliga30.xlsx"
cartoliga = pd.read_excel(path, engine='openpyxl')
print(type(cartoliga))

<class 'pandas.core.frame.DataFrame'>


In [20]:
with pd.ExcelWriter(path, engine="openpyxl", mode = "r") as writer:
    pd_pontos_rodada.to_excel(writer, sheet_name= "Rodada " + rodada)